In [106]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import unicodedata
import os
import webbrowser
import html5lib
from openpyxl import workbook
from datetime import datetime
import requests

In [107]:
path = '../Master_tables_GBDC_Investment.xlsx'
dataframes = pd.read_excel(path, sheet_name=None)

In [142]:
process_tables = {}
process_tables_shape = {}
if not os.path.exists('../PT_csv_file'):
    os.makedirs('../PT_csv_file')


def run_process_function(dataframes, process_tables, process_tables_shape):
    path = '../process_tables_GBDC_Investment.xlsx'
    writer = pd.ExcelWriter(path=path, engine='openpyxl')
    for dataframe in dataframes:
        print(dataframe)
        processed_table = process_table_function(dataframes[dataframe])
        process_tables[dataframe] = processed_table
        process_tables_shape[dataframe] = processed_table.shape
        processed_table.to_excel(
            writer, sheet_name=dataframe.replace(',', ''), index=False)
        processed_table.to_csv(
            '../PT_csv_file/'+dataframe.replace(',', '')+'.csv')
        writer.book.save(path)
    writer.close()

In [143]:
def shape(count, df):
    print(f"{count} : shape : {df.shape}")
    count += 1
    return count


def dropna_col_row(df):
    df = df.dropna(how='all', axis=0).reset_index(drop=True)
    df = df.dropna(how='all', axis=1).reset_index(drop=True)
    return df


def drop_if_contain(pattern, df):
    matching_rows = df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    df = df[~matching_rows]
    return df


def rename_columns(df):
    num_cols = df.shape[1]
    data_col_mapper = dict(
        zip(df.columns.to_list(), [i for i in range(0, num_cols)]))
    df = df.rename(columns=data_col_mapper)
    return df

In [146]:
def process_table_function(soi_table_df):
    count = 1
    count = shape(count, soi_table_df)
    soi_table_df = soi_table_df.replace(
        r'^\s*\$\s*$', '', regex=True).replace(r'\n', '', regex=True)
    soi_table_df = soi_table_df.replace('-', '0')
    soi_table_df = dropna_col_row(soi_table_df)
    soi_table_df = soi_table_df.apply(
        lambda x: x.strip() if isinstance(x, str) else x)
    count = shape(count, soi_table_df)

    # drops all the extra top row
    pattern = r'Net asset value per common share|How We Addressed the Matter in Our Audit'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)
    count = shape(count, soi_table_df)

    # drops all the extra bottom row
    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)
    count = shape(count, soi_table_df)

    # drop all the col name
    pattern = r'(?:Spread\s*Above|cost|Percentage|Above)'
    soi_table_df = drop_if_contain(pattern, soi_table_df)
    pattern = r'^([Tt]otal)'
    soi_table_df = drop_if_contain(pattern, soi_table_df)
    count = shape(count, soi_table_df)

# drop nan col row
    soi_table_df = dropna_col_row(soi_table_df)
    count = shape(count, soi_table_df)
# drops the sub total
    soi_table_df = soi_table_df.dropna(subset=[soi_table_df.columns[0]])
    count = shape(count, soi_table_df)


# rename col
    soi_table_df = rename_columns(soi_table_df)

    for index, row in soi_table_df.iterrows():
        for column in soi_table_df.columns:
            pattern = re.compile(r'\([a-zA-Z]\)')
            if str(row[column])[-1] == "+" or '+' in str(row[column]):
                next_column_index = soi_table_df.columns.get_loc(
                    column) + 1
                if str(row[column])[-1] == "+":
                    if (next_column_index < len(soi_table_df.columns)
                        and not pd.isna(row[soi_table_df.columns[next_column_index]])
                        ):
                        soi_table_df.at[index, column] = str(
                            soi_table_df.at[index, column])+str(row[soi_table_df.columns[next_column_index]])
                        soi_table_df.at[index,
                                        soi_table_df.columns[next_column_index]] = np.nan

                    next_column_index = soi_table_df.columns.get_loc(
                        next_column_index) + 1
                if (
                    next_column_index < len(soi_table_df.columns)
                    and pattern.search(str(row[soi_table_df.columns[next_column_index]]))
                    and not pd.isna(row[soi_table_df.columns[next_column_index]])
                ):
                    soi_table_df.at[index, column] = str(
                        soi_table_df.at[index, column])+str(row[soi_table_df.columns[next_column_index]])
                    soi_table_df.at[index,
                                    soi_table_df.columns[next_column_index]] = np.nan

            if row[column] == "No Value":
                pattern = re.compile(r'\([0-9]\)')
                next_column_index = soi_table_df.columns.get_loc(column) + 1
                if (
                    next_column_index < len(soi_table_df.columns)
                    and pattern.search(str(row[soi_table_df.columns[next_column_index]]))
                    and not pd.isna(row[soi_table_df.columns[next_column_index]])
                ):
                    soi_table_df.at[index,
                                    column] = row[soi_table_df.columns[next_column_index]]
                    soi_table_df.at[index,
                                    soi_table_df.columns[next_column_index]] = np.nan

    soi_table_df.insert(0, 'Industy', '')

    for index, row in soi_table_df.iterrows():
        if row.nunique() == 2:
            soi_table_df.at[index, 'Industy'] = row.loc[0]
    soi_table_df['Industy'] = soi_table_df['Industy'].replace('', np.nan)

    col_indices = [0, 1, 2]
    soi_table_df.iloc[:, col_indices] = soi_table_df.iloc[:, col_indices].fillna(
        method='ffill')
    col_indices = [0]
    soi_table_df.iloc[:, col_indices] = soi_table_df.iloc[:,
                                                          col_indices].fillna('No value')

    for index, row in soi_table_df.iterrows():
        cleanedList = [x for x in list(row) if str(x) != 'nan']
        row = pd.Series(cleanedList)
        soi_table_df.loc[index] = row


# drop nan col row
    # soi_table_df = soi_table_df.dropna(axis=0, thresh=4)
    soi_table_df = dropna_col_row(soi_table_df)
    count = shape(count, soi_table_df)
# rename col
    soi_table_df = rename_columns(soi_table_df)

    return soi_table_df


run_process_function(dataframes=dataframes, process_tables=process_tables,
                     process_tables_shape=process_tables_shape)

March 31 2013
1 : shape : (424, 31)
2 : shape : (424, 15)
3 : shape : (424, 15)
4 : shape : (424, 15)
5 : shape : (399, 15)
6 : shape : (399, 10)
7 : shape : (365, 10)


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_22062/2457978013.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


8 : shape : (365, 10)
June 30 2013
1 : shape : (432, 31)
2 : shape : (432, 23)
3 : shape : (432, 23)
4 : shape : (431, 23)


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_22062/2457978013.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


5 : shape : (403, 23)
6 : shape : (403, 18)
7 : shape : (365, 18)
8 : shape : (365, 10)
September 30 2013
1 : shape : (486, 41)
2 : shape : (486, 17)
3 : shape : (454, 17)
4 : shape : (440, 17)
5 : shape : (420, 17)
6 : shape : (420, 10)
7 : shape : (381, 10)


...

5 : shape : (1696, 28)
6 : shape : (1696, 24)
7 : shape : (1620, 24)
8 : shape : (1620, 13)
March 31 2023
1 : shape : (1605, 76)
2 : shape : (1605, 28)
3 : shape : (1605, 28)
4 : shape : (1588, 28)


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_22062/2457978013.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


5 : shape : (1534, 28)
6 : shape : (1534, 23)
7 : shape : (1460, 23)
8 : shape : (1460, 13)
June 30 2023
1 : shape : (1523, 76)
2 : shape : (1523, 28)
3 : shape : (1523, 28)
4 : shape : (1504, 28)


/var/folders/mf/yrkcqqr56t955_zz9p9f4swc0000gn/T/ipykernel_22062/2457978013.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)


5 : shape : (1454, 28)
6 : shape : (1454, 23)
7 : shape : (1383, 23)
8 : shape : (1383, 13)


In [148]:
process_tables_shape

{'March 31 2013': (365, 10),
 'June 30 2013': (365, 10),
 'September 30 2013': (381, 10),
 'December 31 2013': (378, 10),
 'March 31 2014': (399, 10),
 'June 30 2014': (412, 10),
 'September 30 2014': (432, 10),
 'December 31 2014': (430, 10),
 'March 31 2015': (445, 10),
 'June 30 2015': (473, 10),
 'September 30 2015': (483, 10),
 'December 31 2015': (425, 10),
 'March 31 2016': (505, 10),
 'June 30 2016': (469, 10),
 'September 30 2016': (582, 10),
 'December 31 2016': (579, 10),
 'March 31 2017': (606, 10),
 'June 30 2017': (635, 10),
 'December 31 2017': (663, 10),
 'March 31 2018': (699, 10),
 'June 30 2018': (737, 10),
 'September 30 2018': (778, 10),
 'December 31 2018': (841, 10),
 'March 31 2019': (875, 10),
 'June 30 2019': (917, 10),
 'September 30 2019': (1013, 10),
 'December 31 2019': (1063, 10),
 'March 31 2020': (1106, 11),
 'June 30 2020': (1105, 10),
 'September 30 2020': (1138, 10),
 'December 31 2020': (1175, 10),
 'March 31 2021': (1227, 10),
 'June 30 2021': (130